## Librerías usadas

En este cuaderno usaremos ``numpy`` y ``matplotlib``. Esta celda importa estos paquetes y define que la salida de las gráficas se muestre en el cuaderno al correr la celda.

In [1]:
#Importamos algunas librerías
import matplotlib.pyplot as plt
import numpy as nmp
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.sans-serif'] = ['Times']
%matplotlib inline

## Solución numérica de Ecuaciones Diferenciales Parciales de segundo orden
$$ \newcommand{\pap}[1]{\left( #1 \right)}
\newcommand{\pas}[1]{\left[ #1 \right]}
\newcommand{\pac}[1]{\left\{ #1 \right\}}
\newcommand{\pav}[1]{\left\vert #1 \right\vert}
\newcommand{\paV}[1]{\left\Vert #1 \right\Vert}
\newcommand{\yb}{\mathbf{y}}
\newcommand{\ub}{\mathbf{u}}
\newcommand{\zzb}{\mathbf{z}}
\newcommand{\Am}{\mathbb{A}}
$$
### Problemas con condiciones de frontera: Diferencias Finitas

Supongamos que queremos solucionar la siguiente ecuación diferencial parcial en dos dimensiones (ecuación de Poisson):
$$
\frac{\partial^2 f}{\partial x^2}+\frac{\partial^2 f}{\partial y^2}=g(x,y)\;.
$$
